In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
google_data = pd.read_csv("google-patent-data.csv")
google_data.head()

In [ ]:
scraped_data = pd.read_csv("scraped_patents.csv")
scraped_data.head()

In [ ]:
google_data.shape

In [ ]:
scraped_data.shape

In [ ]:
patents = pd.merge(google_data, scraped_data, left_on='result link', right_on='url')
patents.head()

In [ ]:
patents_cleaned = patents.drop(columns=['id','result link', 'representative figure link', 'flesh.brix_text'])
patents_cleaned.head()

In [ ]:
patents_cleaned.dtypes

In [ ]:
patents_cleaned['grant date'] = pd.to_datetime(patents_cleaned['grant date'])

patents_cleaned['grant year'] = patents_cleaned['grant date'].dt.year

patents_cleaned.head()

In [ ]:
year_column = patents_cleaned.pop('grant year')

In [ ]:
patents_cleaned.insert(1, 'year', year_column)

In [ ]:
patents_cleaned.head()

In [ ]:
patents_cleaned['pluot name'] = patents_cleaned['title'].str.extract(r'[\'"`“”‘’](.*?)[\'"`“”‘’]', expand=False)

In [ ]:
patents_cleaned.head()

In [ ]:
patents_cleaned['pluot name'] = patents_cleaned['pluot name'].str.title()

In [ ]:
patents_cleaned['pluot name'].to_string(index=False)

In [ ]:
name_column = patents_cleaned.pop('pluot name')

In [ ]:
patents_cleaned.insert(1, 'pluot name', name_column)

In [ ]:
patents_cleaned.head()

In [ ]:
patents_cleaned.to_csv('patents_cleaned.csv', index=False)

In [ ]:
handpicked = pd.read_csv("handpicked-data.csv")
handpicked.head()

In [ ]:
pluots_all = pd.merge(patents_cleaned, handpicked, left_on='pluot name', right_on='Varietal', how='outer')
pluots_all.head()

In [ ]:
pluots_all.shape

In [ ]:
pluots_all['pluot name'] = pluots_all['pluot name'].combine_first(pluots_all['Varietal'])

In [ ]:
pluots_all.drop(columns='Varietal')

In [ ]:
pluots_all.head()

In [ ]:
pluots_all.to_csv("pluots_all.csv", index=False)

In [ ]:
pluots_cleaned = pd.read_csv("pluots_cleaned.csv")
pluots_cleaned.head()

In [ ]:
pluots_cleaned.dtypes

In [ ]:
pluots_cleaned['year'] = pluots_cleaned['year'].astype('Int64')

In [ ]:
pluots_cleaned['year'].value_counts().sort_index().plot(kind='bar')

In [ ]:
pluots_year_brix = pluots_cleaned[['pluot name', 'year', 'scraped_brix']]
pluots_year_brix.head()

In [ ]:
pluots_year_brix.plot.scatter(x='year', y='scraped_brix')

In [ ]:
pluots_taste = pd.read_csv("pluot_taste_scores.csv")
pluots_taste.head()

In [ ]:
pluots_taste.rename(columns={'name': 'pluot name'}, inplace=True)

In [ ]:
pluots_taste.plot(kind='scatter', x='acid', y='flavor')

In [ ]:
pluots_taste.plot(kind='scatter', x='sugar', y='flavor')

In [ ]:
pluots_taste.plot(kind='scatter', x='flavor', y='overall')

In [ ]:
pluots_taste_means = pluots_taste.groupby('pluot name')[['acid', 'sugar', 'flavor', 'overall']].mean()
pluots_taste_means.head()

In [ ]:
pluots_taste_merged = pd.merge(pluots_taste_means, pluots_cleaned[['pluot name', 'scraped_brix']], how='left', on='pluot name')
pluots_taste_merged.head(10)

In [ ]:
pluots_taste_merged.plot(kind='scatter', x='scraped_brix', y='overall')

In [ ]:
pluots_taste_merged.plot(kind='scatter', x='sugar', y='flavor')

# To look at pluot colors

In [ ]:
import ast

# from pluots_cleaned skin.hex and flesh.hex

# Assuming the column with the hex lists is called 'colors'
def extract_hexes(val):
    try:
        parsed = ast.literal_eval(val)
        return [color for color in parsed if color is not None]
    except (ValueError, SyntaxError):
        return []

# Flatten the entire column into one list of hex values
all_skin_hexes = pluots_cleaned['skin.hex'].apply(extract_hexes).sum()
all_flesh_hexes = pluots_cleaned['flesh.hex'].apply(extract_hexes).sum()

In [ ]:
all_skin_hexes

In [ ]:
all_flesh_hexes

In [ ]:
import seaborn as sns

sns.set_theme()

def hex_to_rgb(hex_value):
  h = hex_value.lstrip('#')
  return tuple(int(h[i:i + 2], 16) / 255.0 for i in (0, 2, 4))

all_skin_rgb = list(map(hex_to_rgb, all_skin_hexes))

row_size = 37
rows = [all_skin_rgb[i:i + row_size] for i in range(0, len(all_skin_rgb), row_size)]

for row in rows:
  sns.palplot(row)

In [ ]:
all_flesh_rgb = list(map(hex_to_rgb, all_flesh_hexes))

row_size = 40
rows = [all_flesh_rgb[i:i + row_size] for i in range(0, len(all_flesh_rgb), row_size)]

for row in rows:
  sns.palplot(row)